In [4]:
library(readr) #if not already installed, install.packages("readr")
library(openxlsx) #if not already installed, install.packages("openxlsx")
library(dplyr) # etc.
library(ggplot2)
library(ggrepel)
library(gridExtra)
library(viridis)


Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine



In [6]:
#main
#load in the first timeframe's data - in this case, Obama era counts
first <- read_csv("https://raw.githubusercontent.com/edgi-govdata-archiving/web_monitoring_research/r/data/obama_count_r.csv", col_names = FALSE) 
#load in the second timeframe's data - in this case, Trump era counts
second <- read_csv("https://raw.githubusercontent.com/edgi-govdata-archiving/web_monitoring_research/r/data/trump_count_r.csv", col_names = FALSE) 

#convert missing values (999) due to WM error to NAs
#first[first==999] <- NA
#second[second==999] <- NA

#load in a CSV that has the list of terms - this is for formatting outputs
terms<-read_csv("https://raw.githubusercontent.com/edgi-govdata-archiving/web_monitoring_research/r/data/terms.csv", col_names = FALSE) 
terms<-tolower(terms)

colnames(first)<-terms
colnames(second)<-terms

urls <- read_csv("https://raw.githubusercontent.com/edgi-govdata-archiving/web_monitoring_research/r/data/counted_urls.csv") 
#slashes removed means trailing slashes - those at the end of some urls - have been removed #("inputs/counted_urls.csv") #load in a CSV that has a list of the URLs, organizations, etc. #STANDARD SLASHES
#_slashes_removed

## RECOUNTS
# add in recounts - OBAMA
recounts <- read_csv("https://raw.githubusercontent.com/edgi-govdata-archiving/web_monitoring_research/r/data/2016BLM-DOE-OSHA_counts.csv", col_names = FALSE)
tt<-append(terms, "index", after=0)
colnames(recounts)<-tt
for (org in c("DOE", "BLM", "OSHA")){
  urls_recounts<-which(urls$org==org)
  recounts_selected<-recounts[recounts$index %in% urls_recounts,] #subset to selected org
  first[c(urls_recounts),]<-recounts_selected[,2:57]
}
# Do a manual recount for selected DOE pages and term ('efficiency'). Extra 4 uses on many DOE pages in Obama era.
# find urls that include "energy.gov/eere"
urls.manual<-grep('energy.gov/eere', urls$`url - o`)
# subtract 4 for navbar and footer counts
first[c(urls.manual),'efficiency']<-first[c(urls.manual),'efficiency']-4

# add in recounts - TRUMP
recounts <- read_csv("https://raw.githubusercontent.com/edgi-govdata-archiving/web_monitoring_research/r/data/2018BLM-DOE-OSHA_counts.csv", col_names = FALSE)
tt<-append(terms, "index", after=0)
colnames(recounts)<-tt
for (org in c("DOE", "BLM", "OSHA")){
  urls_recounts<-which(urls$org==org)
  recounts_selected<-recounts[recounts$index %in% urls_recounts,] #subset to selected org
  second[c(urls_recounts),]<-recounts_selected[,2:57]
} 
write_csv(first, "obama_count.csv")
write_csv(second, "trump_count.csv")
# delete the above RECOUNTS and just load it....

Parsed with column specification:
cols(
  .default = col_integer()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_integer()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  `url - o` = col_character(),
  `shortened url - o` = col_character(),
  `captured url - 0` = col_character(),
  `url - t` = col_character(),
  `shortened url - t` = col_character(),
  `final captured url - t` = col_character(),
  domain = col_character(),
  org = col_character()
)
Parsed with column specification:
cols(
  .default = col_integer()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_integer(),
  X1 = col_character()
)
See spec(...) for full column specifications.


In [ ]:
#this does matrix math to calculate, for each page and term, the change (positive, negative, or zero) in usage
combined<-second-first 
combined

In [ ]:
### PROCESSING
## Handle duplicated URLs - find their "index"....
duplicates<-which(duplicated(urls$`url - o`) | duplicated(urls$`url - o`, fromLast = TRUE)) #url duplicates
duplicates.short<-which(duplicated(urls$`shortened url - o`) | duplicated(urls$`shortened url - o`, fromLast = TRUE))#shortened url duplicates
duplicates<-c(duplicates, duplicates.short)
duplicates<-unique(duplicates)

#....and then filter away
first<-first[-c(duplicates),]
second<-second[-c(duplicates),]
combined<-combined[-c(duplicates),]
urls<-urls[-c(duplicates),]

## Handle pages we don't want (e.g. Jan 2017 snapshots, blogs, Spanish-language pages)
pages.snapshots<-grep('snapshot', urls$`url - o`)
pages.edu<-grep('edu/', urls$`url - o`)
pages.news<-grep('news', urls$`url - o`)
pages.blog<-grep('blog', urls$`url - o`)
pages.News<-grep('News', urls$`url - o`)
pages.espanol<-grep('espanol', urls$`url - o`)

pages.length<-sapply(gregexpr("/", urls$`url - o`), length) # gets the number of slashes in each url (a proxy for importance/relevance)
pages.length<-which(pages.length > 6) # remove pages greater than 6 slashes. arbitrary, but remember that every url will have at least 3 slashes - http://wwww.epa.gov/.... 2 in the "removed slashes" file
#2987 URLs > 4 3-4 http://www.epa.gov/theme/ http://www.epa.gov/theme http://www.epa.gov/ 
#2314 URLs NOT 5 or NOT 6 5-6  http://www.epa.gov/theme/subtheme/page http://www.epa.gov/theme/subtheme/page/ http://www.epa.gov/theme/subtheme 5 OR 6
#296 URLs 7 < 7 http://www.epa.gov/theme/subtheme/page/subpage http://www.epa.gov/theme/subtheme/page/subpage/subsubpage
#http://www.epa.gov/theme/subtheme/page/ is max
# default is > 6
# for page depth analysis > 4 gets "2 or less" . getting rid of urls with 5 or more slashes. 
# for page depth analysis < = 4 gets 3 or more. getting rid of urls with four or fewer

dump<-c(pages.snapshots,pages.edu, pages.news,pages.blog, pages.News, pages.espanol, pages.length)
dump<-unique(dump)

first<-first[-c(dump),]
second<-second[-c(dump),]
combined<-combined[-c(dump),]
urls<-urls[-c(dump),]

##compare only on snapshots in both timeframes
snaps<-which(!is.na(urls$`captured url - 0`) & !is.na(urls$`final captured url - t`))
first<-first[c(snaps),]
second<-second[c(snaps),]
combined<-combined[c(snaps),]
urls<-urls[c(snaps),]

##compare only on available counts (disregard NAs/999s)
nas<-which(is.na(first[,1])|is.na(second[,1]))
first<-first[-c(nas),]
second<-second[-c(nas),]
combined<-combined[-c(nas),]
urls<-urls[-c(nas),]

In [ ]:
## Optional filter to only climate-related pages
#filter our list to only those pages concerning the "climate" topic - that is, where the Obama version mentioned "climate" "climate change" or "GHG"
#cc<-which(first[,'climate']>0 | first[,'climate change']>0 | first[,'greenhouse gases']>0) # page indices where Obama used these terms
#cc<-which(first[,'climate change']>0) # page indices where Obama used these terms

#first<-first[c(cc),]
#second<-second[c(cc),]
#combined<-combined[c(cc),]
#urls<-urls[c(cc),]

## co-changes/page-specific changes for termX->termY
#Pages where costs increased and benefits decreased
#Pages where emissions decreased and air quality increased
#pages where climate change decreased and resilience increased
#pages where climate change decreased and sustainability increased

#filter<-which(combined[,'climate']>0 & combined[,'climate change']<0)
#first.filter<-first[c(filter),]
#second.filter<-second[c(filter),]
#combined.filter<-combined[c(filter),]
#urls.filter<-urls[c(filter),]

In [ ]:
#### DEBUG
##verify obama and trump urls match
errors=0
for (i in 1:nrow(urls)) {
  if (tolower(urls$`shortened url - o`[i]) != tolower(urls$`shortened url - t`[i])){
    errors<-errors+1
    print(i)
  }
}
errors

In [ ]:
##### FIGURE 4
# again, this could be done in a loop to compare multiple terms, but for our purposes, we'll look at "climate change"
i=9

#"Delta chart" by term...FIGURE PG 14
df<-data.frame(first[,i], second[,i])
colnames(df)<-c("obama", "trump")
change<-which(df$obama != 0 | df$trump != 0) #REMOVE ZEROS - we don't want to plot pages that didn't change
df<-df[c(change),]
df$change <- df$trump-df$obama
df.unique<-df %>% group_by(obama, change)%>%mutate(chg = sum(change), count=n()) #%>%mutate(count = n()) # count of unique count pairings by URLs. (1,2 | 1,4 | 3,1 | etc.) How many of each change 1->2, 1->4, 3->1 are there?
df.unique<-unique(df.unique)
pal <- c("red", "blue")
p<-ggplot(df.unique, aes(obama, change, alpha=count, fill=change>0)) +
geom_bar(stat='identity', position='dodge') +
scale_fill_manual(values=pal) +
labs(title = 'Changes to the use of “climate change”', x='Count of "climate change" in 2016', y='Difference in 2018')+
theme(text=element_text(size=10), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        plot.background=element_rect(fill='white'), panel.background = element_rect(fill='white'), 
        legend.position="none", axis.line.x = element_line(colour = "black", size=0)) +
scale_y_continuous(limits=c(-40,20), breaks=seq(-100, 80, 5)) +
scale_x_continuous(limits=c(0,35), breaks=seq(0, 150, 5)) + 
geom_hline(yintercept=0, linetype="dashed", color = "black")

In [ ]:
##### FIGURE 5
# again, this could be done in a loop to compare multiple terms, but for our purposes, we'll look at "climate change"
# and resilience
i=9
j=43
#assess term pairing in first
print(terms[c(i,j)]) #the terms we're working on right now
col<-first[c(i,j)] #from the first timeframe, the term count
tcol<-second[c(i,j)] #from the second teimframe, the term count
#colcombined<-combined[i] #second - first count, for this term

urlsDump<-urls # a temporary variable for our URLs
orgsDump<-data.frame(urls$org) # a temporary variable for our URLs

#shed zeros - We won't count pages if both terms weren't used on same page
zeros<-which(col[,1]==0 & col[,2]==0)
col<-col[-c(zeros),]
tcol<-tcol[-c(zeros),] #here we are saying that we will eliminate second timeframe urls if in the first time the urls did not have both terms used
urlsDump<-urlsDump[-c(zeros),1]
orgsDump<-orgsDump[-c(zeros),1]

#Here we bring together, for the current term, the first era, second era, and combined counts, for outputting as a CSV
combo<-cbind(urlsDump, col, tcol, orgsDump)
colnames(combo)<-c("url", "o_CC", "o_R", "t_CC", "t_R", "orgs")

#now subset data and draw arrows
#change<-which(combo[,2]-combo[,4]!= 0 & combo[,3]-combo[,5]!= 0) #NOTE: BOTH TERMS HAVE TO CHANGE....i.e. no flat lines. change?
dir<-which(combo[,2]-combo[,4]>0 & combo[,3]-combo[,5] <0)
#NE: dir<-which(combo[,2]-combo[,4]<0 & combo[,3]-combo[,5] <0)
#SW: dir<-which(combo[,2]-combo[,4]>0 & combo[,3]-combo[,5] >0)
#SE (resilience decrease, CC increase): dir<-which(combo[,2]-combo[,4]<0 & combo[,3]-combo[,5] >0)
#NW: dir<-which(combo[,2]-combo[,4]>0 & combo[,3]-combo[,5] <0)
#E:   dir<-which(combo[,2]-combo[,4]<0 & combo[,3]-combo[,5] == 0) 
#N: dir<-which(combo[,2]-combo[,4]==0 & combo[,3]-combo[,5] < 0) 
#S: dir<-which(combo[,2]-combo[,4]==0 & combo[,3]-combo[,5] > 0) 
combo<-combo[c(dir),] #dir orgfilterchange
#combo$Dir<-ifelse(combo[,2]-combo[,4]>0, "NE", "NE")
Legend<-factor(combo$Dir)
#Legend<-factor(combo$orgs)
#orgfilter<-filter(combo, orgs=='WH')
#combo<-orgfilter

combo<-combo[,c(2:5)]
c<-combo %>% group_by(o_CC, o_R, t_CC, t_R) %>%mutate(count = n())
c<-unique(c)
#f<-factor(c$count/10)

p<-ggplot(c,aes(x = o_CC,y = o_R))+
geom_point(aes(x = o_CC,y = o_R), size=0, color="white") + 
geom_point(aes(x = t_CC,y = t_R), size=0, color="white") +
scale_y_continuous(limits=c(0,22), expand = c(0,0)) +
scale_x_continuous(limits=c(0,22),expand = c(0,0)) + 
labs(x="Count of climate change", y="Count of resilience")+
theme(aspect.ratio=1)

p + theme(text=element_text(size=9), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), legend.position="none", 
        axis.line = element_line(colour = "grey68", size=.1)) +
geom_segment(aes(x = o_CC,y = o_R,xend = t_CC,yend = t_R, size=3), color="black", arrow=arrow(angle = 30, length = unit(0.12, "inches"), ends = "last", type = "open")) + 
geom_abline(intercept=0, slope=1, colour="grey68", linetype=2, size=.2) + scale_size(range = c(.1, 1.2),guide=FALSE)  #+ scale_color_manual(values=c("#ef8a62", "#67a9cf"))#+  scale_color_brewer(palette="Paired")#+geom_text(aes(label=url),hjust=0, vjust=0, size = 2)

In [ ]:
#### FIGURE 6

by.org<-urls%>% group_by(org) %>%summarise(num_of_pages=n()) # summarize all URLs by org (agency)

# This could be configured as a loop to go through each term but for our purposes here, we'll only look at "climate change"

i=9

#report by org and count
#pctchg<-unlist(combined[i]/first[i])
pc<-which(is.na(combined[i]/first[i])) 
combined.url<-cbind(urls[c(1,3,6,8)], first[i], second[i], combined[i]) #combined.url is only the urls that aren't NA after combined/first. In other words, we exclude 0/0 (0 obama, 0 change) #how many pages was the term on in at least one of the timeframes...
combined.url<-combined.url[-c(pc),]
colnames(combined.url)<-c("url", "obama wm", "trump wm", "org", "before", "after","diff")

# debug/fishy check
fishy.after<-combined.url%>%group_by(after,org)%>%mutate(pages=n())%>%distinct(after,org,pages)
fishy.after<-merge(fishy.after, by.org, by='org')

# print list of counts
listOfCounts<-cbind(terms[i],sum(combined.url$before), sum(combined.url$after), sum(combined.url$diff), 100*(sum(combined.url$diff)/sum(combined.url$before)), nrow(combined.url)) #sum(first[i]), sum(second[i]), sum(combined[i]), 100*(sum(combined[i])/sum(first[i])))
#diganostic<-cbind(first[i], second[i], combined[i], urls$`captured url - 0`, urls$`final captured url - t`)

# change by term
stats<-combined.url%>% group_by(org) %>%summarise(ObamaSum=sum(before), TrumpSum=sum(after), overall_pct_chg = 100*(sum(diff)/sum(before)), OCount=sum(before >0 ), TCount=sum(after>0), num_of_pages=n())
stats<-merge(stats, by.org, by = "org")
stats$pct<-(stats$num_of_pages.x/stats$num_of_pages.y)*100
stats$pagepctchg<-((stats$TCount-stats$OCount)/stats$OCount)*100

# Put together the full term count including 0s
full<-cbind(urls[c(1,3,6,8)], first[i], second[i], combined[i], 100*(combined[i]/first[i]))
colnames(full)<-c("url", "obama wm", "trump wm", "org", "before", "after","diff","pctdiff")
full<-merge(full, obamaDocLength, by="url", all.x=TRUE)
full<-merge(full, trumpDocLength, by="url", all.x=TRUE)

#visualize term data by agency - changes in average use
#calculate obama rate of use, trump rate of use
stats.viz<-stats
stats.viz$Obamadensity<-stats.viz$ObamaSum/stats.viz$num_of_pages.y #rate of use across all agency pages
stats.viz$Obamaintensity<-stats.viz$ObamaSum/stats.viz$num_of_pages.x #rate of use across the pages it's used on
stats.viz$Trumpdensity<-stats.viz$TrumpSum/stats.viz$num_of_pages.y #rate of use across all agency pages
stats.viz$Trumpintensity<-stats.viz$TrumpSum/stats.viz$num_of_pages.x #rate of use across the pages it's used on

stats.viz$position[stats.viz$TrumpSum- stats.viz$ObamaSum == 0] = "No Change"
stats.viz$position[stats.viz$TrumpSum- stats.viz$ObamaSum > 0] = "Increase"
stats.viz$position[stats.viz$TrumpSum- stats.viz$ObamaSum < 0] = "Decrease"
stats.viz$position[stats.viz$TrumpSum- stats.viz$ObamaSum == 0] = "No Change"

#FIGURE 6
title<-("Climate Change - Average Per Page Use by Agency")
#file<-paste("outputs/figures/by term/",title,".png", sep="")  #save
p<-ggplot(stats.viz,aes(x = Obamadensity,y = Trumpdensity, label = stats.viz$org))+
geom_abline(intercept=0, slope=1, colour="black", size=.3, linetype="dashed") +
geom_point(aes(x = Obamadensity,y = Trumpdensity,  color=stats.viz$position), size=4) + 
scale_x_sqrt(breaks=c(0, .1,.25, .5, 1, 2, 5, 10)) +
scale_y_sqrt(breaks=c(0, .1,.25, .5, 1,2, 5, 10)) +
labs(x="2016", y="2018", title=title) +
geom_text_repel(colour='dark grey', size = 4, box.padding = .75) + 
scale_colour_manual(values = c("Red", "Blue", "Grey"))

p + theme(text=element_text(size=10), panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
         plot.background=element_rect(fill='white'), panel.background = element_rect(fill='white'), 
         legend.position="none", axis.line = element_line(colour = "black", size=0)) 
#+ scale_color_manual(values=c("#ef8a62", "#67a9cf"))#+  scale_color_brewer(palette="Paired")#+geom_text(aes(label=url),hjust=0, vjust=0, size = 2)
# ggsave(file, plot=last_plot())

In [ ]:
#line graph FIGURE 7
# ideally the creation of cabinet_counts and cabinet_average would be done _in this script_

f<-read_csv("outputs/cabinet_average.csv")
f$Term<-factor(f$Term, levels=c("climate change", "climate", "greenhouse gases", "clean energy", 
                                "hydraulic fracturing", "adaptation", "air quality", "emissions", 
                                "resilience", "sustainability", "unconventional gas", "unconventional oil", 
                                "energy independence"))

p<-ggplot(data=f, aes(x=Term, y=Average, group=Cabinet)) +
  geom_line(aes(color=Cabinet), size=2)+
  geom_point(aes(color=Cabinet), size=5) + 
  geom_abline(intercept=0, slope=0, colour="black", size=.1) +
  theme(text=element_text(size=12), 
        axis.text.x = element_text(angle = 45, hjust = 1), 
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(),
        panel.background = element_blank(), 
        legend.position="none",
        axis.line = element_line(colour = "grey68", size=1)) +
  labs(title="Cabinet vs. Non-Cabinet Average Change Per Page", y="Change") +
  scale_color_manual(values=c('#5ab4ac','#d8b365')) +
  annotate("text", label = "Cabinet", x = 1, y = -3, size = 5, colour = "black")  +
  annotate("text", label = "Non-Cabinet", x = 1.5, y = .5, size = 5, colour = "black")
p

In [ ]:
#box and whiskers FIGURE 8
# ideally the creation of cabinet_counts and cabinet_average would be done _in this script_

f <- read_csv("outputs/cabinet_counts.csv")
f$term <- factor(f$term, levels=c("climate change", "climate", 
                                        "greenhouse gases", "clean energy", "hydraulic fracturing", 
                                        "adaptation", "air quality", "emissions", "resilience", "sustainability", 
                                        "unconventional gas", "unconventional oil", "energy independence"))

#How many of each change 1->2, 1->4, 3->1 are there?
f <- f %>% group_by(term, change, cabinet)%>%mutate(count=n())

p <- ggplot(f, aes(x=term, y=change)) +
  #geom_dotplot(aes(color=cabinet), binaxis='y', stackdir='center', binwidth = 20, dotsize=.5, position=position_dodge(1)) +
  geom_point(aes(shape="20", color=cabinet, size=count), na.rm=TRUE, position=position_dodge(.5))+
  scale_size(range = c(3, 10)) +
  #stat_boxplot(geom = 'errorbar') +
  #geom_boxplot(position=position_dodge(.5)) +
  #scale_y_continuous(limits=c(-25,25))
  geom_abline(intercept=0, slope=0, colour="black", size=.1) +
  theme(text=element_text(size=14), axis.text.x = element_text(angle = 45, hjust = 1),
          panel.grid.major = element_blank(), 
          panel.grid.minor = element_blank(),
          panel.background = element_blank(), 
          legend.position="none", 
          axis.line = element_line(colour = "grey68", size=.1)) +
  labs(title="Cabinet vs. Non-Cabinet Per Page Changes", y="Change", x="Term") +
  scale_color_manual(values=c('#d8b365','#5ab4ac')) +
  annotate("text", angle = 45, hjust=1, label = "Cabinet", x = 1, y = 15, size = 3, colour = "#d8b365")  +
  annotate("text", angle = 45, hjust=1, label = "Non-Cabinet", x = 1.5, y = 30, size = 3, colour = '#5ab4ac')
p

In [ ]:
#FIGURE 9
x <- read_csv("outputs/depth/depthR.csv")
colnames(x)<-c("Term", "Visibility", "Percent", "Viz")
x <- x[order(-x$Percent),]
pal <- c("red", "blue")
ggplot(x, aes(x=Visibility, y=Percent, alpha=Viz, fill = Percent>0)) + 
  geom_bar(position="dodge", stat="identity", width = .5) +
  scale_fill_manual(values = pal) +
  geom_hline(yintercept=0, size=.5, linetype="dashed") +
  ggtitle("#Differences in Use of 'Climate Change' by Agency and Page Visibility") + 
  facet_wrap(~Agency, ncol=3) + #Agency
  coord_flip() +
  xlab("") +  
  scale_x_discrete(breaks=NULL, labels=NULL)+
  theme(text=element_text(size=10),panel.background = element_rect(fill='white'), 
        legend.position="none", axis.text.y=element_blank(), panel.grid.major.y = element_blank(), 
        panel.grid.minor.y=element_blank(), panel.grid.minor.x=element_blank(), panel.grid.major.x=element_blank()) 

In [ ]:
#FIGURE 10
x <- read_csv("outputs/depth/depthAgencyR.csv") 
x<-first

colnames(x)<-c("Agency", "Visibility", "Percent", "Viz") 
x <- x[order(-x$Percent),]
pal <- c("red", "blue")
ggplot(x, aes(x=Visibility, y=Percent, alpha=Viz, fill = Percent>0)) + 
  geom_bar(position="dodge", stat="identity", width = .5) +
  scale_fill_manual(values = pal) +
  geom_hline(yintercept=0, size=.5, linetype="dashed") +
  ggtitle("Differences in Use of Terms by Page Visibility") + 
  facet_wrap(~Agency, ncol=3) + #Agency
  coord_flip() +
  xlab("") +  
  scale_x_discrete(breaks=NULL, labels=NULL)+
  theme(text=element_text(size=10),panel.background = element_rect(fill='white'), 
        legend.position="none", axis.text.y=element_blank(), panel.grid.major.y = element_blank(), 
        panel.grid.minor.y=element_blank(), panel.grid.minor.x=element_blank(), panel.grid.major.x=element_blank()) 